# 第4章: データと指示の分離

- [レッスン](#lesson)
- [演習](#exercises)
- [サンプル実験場](#example-playground)

## セットアップ

以下のセットアップセルを実行して、APIキーを読み込み、`get_completion`ヘルパー関数を確立してください。

In [ ]:
# Import python's built-in regular expression library
import re
import boto3
import json

# Import the hints module from the utils package
import os
import sys
module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import hints

# Retrieve the MODEL_NAME variable from the IPython store
%store -r MODEL_NAME
%store -r AWS_REGION

client = boto3.client('bedrock-runtime',region_name=AWS_REGION)

def get_completion(prompt,system=''):
    body = json.dumps(
        {
            "anthropic_version": '',
            "max_tokens": 2000,
            "messages": [{"role": "user", "content": prompt}],
            "temperature": 0.0,
            "top_p": 1,
            "system": system
        }
    )
    response = client.invoke_model(body=body, modelId=MODEL_NAME)
    response_body = json.loads(response.get('body').read())

    return response_body.get('content')[0].get('text')

---

## レッスン

多くの場合、完全なプロンプトを書くのではなく、**後で追加の入力データで修正してからClaudeに送信できるプロンプトテンプレート**が欲しくなります。これは、Claudeに毎回同じことをしてもらいたいが、Claudeがタスクで使用するデータが毎回異なる場合に便利です。

幸い、**プロンプトの固定的な骨組みを変数ユーザー入力から分離し、その後ユーザー入力をプロンプトに代入してから**完全なプロンプトをClaudeに送信することで、これをかなり簡単に行うことができます。

以下では、代入可能なプロンプトテンプレートの書き方と、ユーザー入力の代入方法について、段階的に説明します。

### 例

この最初の例では、Claudeに動物の鳴き声ジェネレーターとして動作するよう求めています。Claudeに送信される完全なプロンプトが、入力（この場合は「Cow」）で代入された`PROMPT_TEMPLATE`だけであることに注目してください。完全なプロンプトを印刷する際に、「Cow」という単語がf-string経由で`ANIMAL`プレースホルダーを置き換えていることに注目してください。

**注意:** 実際には、プレースホルダー変数を特定の名前で呼ぶ必要はありません。この例では`ANIMAL`と呼びましたが、`CREATURE`や`A`と呼ぶこともできました（ただし、一般的には変数名を具体的で関連性のあるものにして、代入なしでもプロンプトテンプレートが理解しやすくなるようにすることが良いでしょう。これはユーザーの解析しやすさのためです）。変数に名前を付ける際は、プロンプトテンプレートのf-stringで使用するものと同じ名前にしてください。

In [ ]:
# Variable content
ANIMAL = "Cow"

# Prompt template with a placeholder for the variable content
PROMPT = f"I will tell you the name of an animal. Please respond with the noise that animal makes. {ANIMAL}"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

なぜこのように入力を分離して代入したいのでしょうか？**プロンプトテンプレートは繰り返しタスクを簡素化します**。あなたがプロンプトにコンテンツを送信する第三者ユーザーを招待するプロンプト構造を構築するとしましょう（この場合、鳴き声を生成したい動物）。これらの第三者ユーザーは、完全なプロンプトを書いたり見たりする必要がありません。彼らがする必要があるのは変数を埋めることだけです。

ここでは変数とf-stringを使用してこの代入を行っていますが、format()メソッドでも行うことができます。

**注意:** プロンプトテンプレートには必要な数だけ変数を持つことができます！

このような代入変数を導入する際、**Claudeに変数がどこで始まりどこで終わるかを確実に知らせる**ことが非常に重要です（指示やタスクの説明と対比して）。指示と代入変数の間に分離がない例を見てみましょう。

人間の目には、以下のプロンプトテンプレートで変数がどこで始まりどこで終わるかは非常に明確です。しかし、完全に代入されたプロンプトでは、その区切りが不明確になります。

In [ ]:
# Variable content
EMAIL = "Show up at 6am tomorrow because I'm the CEO and I say so."

# Prompt template with a placeholder for the variable content
PROMPT = f"Yo Claude. {EMAIL} <----- Make this email more polite but don't change anything else about it."

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

ここで、**Claudeは「Yo Claude」を書き直すべきメールの一部だと思っています**！これは「Dear Claude」で書き直しを始めていることからわかります。人間の目には、特にメールがどこで始まりどこで終わるかがプロンプトテンプレートで明確ですが、代入後のプロンプトでははるかに不明確になります。

どうやってこれを解決するのでしょうか？**入力をXMLタグで囲みます**！以下でこれを行い、ご覧のように、出力にもう「Dear Claude」はありません。

[XMLタグ](https://docs.anthropic.com/claude/docs/use-xml-tags)は`<tag></tag>`のような角括弧タグです。これらはペアで提供され、`<tag>`のような開始タグと、`</tag>`のような`/`でマークされた終了タグから構成されます。XMLタグは、`<tag>content</tag>`のようにコンテンツを囲むために使用されます。

**注意:** Claudeは幅広い区切り文字や区切り記号を認識して作業できますが、Claude専用の区切り文字として**特にXMLタグを使用することをお勧めします**。Claudeは、XMLタグをプロンプト整理メカニズムとして認識するよう特別に訓練されているからです。関数呼び出し以外では、**パフォーマンスを最大限に向上させるためにClaudeが訓練された特別なソースXMLタグはありません**。私たちは意図的にClaudeをこのように非常に柔軟でカスタマイズ可能にしました。

In [ ]:
# Variable content
EMAIL = "Show up at 6am tomorrow because I'm the CEO and I say so."

# Prompt template with a placeholder for the variable content
PROMPT = f"Yo Claude. <email>{EMAIL}</email> <----- Make this email more polite but don't change anything else about it."

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

XMLタグがどのように私たちを助けるかの別の例を見てみましょう。

以下のプロンプトでは、**Claudeがプロンプトのどの部分が指示で、どの部分が入力かを間違って解釈しています**。フォーマットのために、`Each is about an animal, like rabbits`をリストの一部と間違って考えており、ユーザー（`SENTENCES`変数を記入する人）はおそらくそれを望んでいませんでした。

In [ ]:
# Variable content
SENTENCES = """- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs"""

# Prompt template with a placeholder for the variable content
PROMPT = f"""Below is a list of sentences. Tell me the second item on the list.

- Each is about an animal, like rabbits.
{SENTENCES}"""

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

これを修正するには、**ユーザー入力の文をXMLタグで囲む**だけです。これにより、`Each is about an animal, like rabbits.`の前の誤解を招くハイフンにもかかわらず、入力データがどこで始まりどこで終わるかをClaudeに示します。

In [ ]:
# Variable content
SENTENCES = """- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs"""

# Prompt template with a placeholder for the variable content
PROMPT = f""" Below is a list of sentences. Tell me the second item on the list.

- Each is about an animal, like rabbits.
<sentences>
{SENTENCES}
</sentences>"""

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

**注意:** 「Each is about an animal」プロンプトの間違ったバージョンでは、この例で望ましい間違った方法でClaudeに応答させるためにハイフンを含める必要がありました。これはプロンプティングについての重要な教訓です：**小さな詳細が重要です**！**プロンプトのタイプミスや文法エラーをチェックする**ことは常に価値があります。Claudeはパターンに敏感で（初期の頃、ファインチューニング前には生のテキスト予測ツールでした）、あなたが間違いを犯すと間違いを犯しやすく、賢く聞こえるとより賢くなり、ばかげて聞こえるとよりばかげたりします。

上記の内容を変更せずにレッスンプロンプトを試したい場合は、レッスンnotebookの一番下までスクロールして[**サンプル実験場**](#example-playground)をご覧ください。

---

## 演習
- [演習 4.1 - 俳句のトピック](#exercise-41---haiku-topic)
- [演習 4.2 - タイプミスのある犬の質問](#exercise-42---dog-question-with-typos)
- [演習 4.3 - 犬の質問パート2](#exercise-43---dog-question-part-2)

### 演習 4.1 - 俳句のトピック
`PROMPT`を修正して、`TOPIC`という変数を受け取り、そのトピックについての俳句を出力するテンプレートにしてください。この演習は、f-stringを使った変数テンプレート構造の理解をテストすることを目的としています。

In [ ]:
# Variable content
TOPIC = "Pigs"

# Prompt template with a placeholder for the variable content
PROMPT = f""

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("pigs", text.lower()) and re.search("haiku", text.lower()))

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ ヒントが欲しい場合は、以下のセルを実行してください！

In [ ]:
print(hints.exercise_4_1_hint)

### 演習 4.2 - タイプミスのある犬の質問
XMLタグを追加してClaudeが正しい答えを出すように`PROMPT`を修正してください。

プロンプトの他の部分は変更しないようにしてください。乱雑で間違いだらけの文章は意図的なもので、そのような間違いに対してClaudeがどのように反応するかを見ることができます。

In [ ]:
# Variable content
QUESTION = "ar cn brown?"

# Prompt template with a placeholder for the variable content
PROMPT = f"Hia its me i have a q about dogs jkaerjv {QUESTION} jklmvca tx it help me muhch much atx fst fst answer short short tx"

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("brown", text.lower()))

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ ヒントが欲しい場合は、以下のセルを実行してください！

In [ ]:
print(hints.exercise_4_2_hint)

### 演習 4.3 - 犬の質問パート2
XMLタグを追加**せずに**`PROMPT`を修正してください。代わりに、プロンプトから1つまたは2つの単語だけを削除してください。

上記の演習と同様に、プロンプトの他の部分は変更しないようにしてください。これにより、Claudeがどのような言語を解析し理解できるかがわかります。

In [ ]:
# Variable content
QUESTION = "ar cn brown?"

# Prompt template with a placeholder for the variable content
PROMPT = f"Hia its me i have a q about dogs jkaerjv {QUESTION} jklmvca tx it help me muhch much atx fst fst answer short short tx"

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("brown", text.lower()))

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ ヒントが欲しい場合は、以下のセルを実行してください！

In [ ]:
print(hints.exercise_4_3_hint)

### おめでとうございます！

この時点まですべての演習を解決した場合、次の章に進む準備ができています。楽しいプロンプティングを！

---

## サンプル実験場

これは、このレッスンで示されたプロンプトの例を自由に実験し、プロンプトを調整してClaudeの応答にどのような影響を与えるかを確認するエリアです。

In [ ]:
# Variable content
ANIMAL = "Cow"

# Prompt template with a placeholder for the variable content
PROMPT = f"I will tell you the name of an animal. Please respond with the noise that animal makes. {ANIMAL}"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# Variable content
EMAIL = "Show up at 6am tomorrow because I'm the CEO and I say so."

# Prompt template with a placeholder for the variable content
PROMPT = f"Yo Claude. {EMAIL} <----- Make this email more polite but don't change anything else about it."

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# Variable content
EMAIL = "Show up at 6am tomorrow because I'm the CEO and I say so."

# Prompt template with a placeholder for the variable content
PROMPT = f"Yo Claude. <email>{EMAIL}</email> <----- Make this email more polite but don't change anything else about it."

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# Variable content
SENTENCES = """- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs"""

# Prompt template with a placeholder for the variable content
PROMPT = f"""Below is a list of sentences. Tell me the second item on the list.

- Each is about an animal, like rabbits.
{SENTENCES}"""

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# Variable content
SENTENCES = """- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs"""

# Prompt template with a placeholder for the variable content
PROMPT = f""" Below is a list of sentences. Tell me the second item on the list.

- Each is about an animal, like rabbits.
<sentences>
{SENTENCES}
</sentences>"""

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))